In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 3, Finished, Available, Finished)

###### **<mark>Creating Gold_Product table</mark>**

In [2]:
%%sql
create table if not exists Ecommerce_project.gold_product
(
    Product_ID int,
    Product_Category string,
    Product string,
    Created_TS TIMESTAMP,
    Modified_TS TIMESTAMP
)
USING DELTA

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

###### **<mark>Building logic for incremental load</mark>**

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Ecommerce_project.gold_product").first()[0]

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 5, Finished, Available, Finished)

###### **<mark>Creating dataframe with required columns for gold Custimer table</mark>**

In [4]:
df_bronze=spark.sql("select distinct Product_Category, Product from Ecommerce_project.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 6, Finished, Available, Finished)

###### **<mark>Creating Product_ID column using montonically_increasing_id</mark>**

In [5]:
Max_ID = spark.sql("select coalesce(max(Product_ID),0) from Ecommerce_project.gold_product").first()[0]

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 7, Finished, Available, Finished)

In [6]:
final_df= df_bronze.withColumn('Product_ID',monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 8, Finished, Available, Finished)

###### **<mark>Creating view from final_df</mark>**

In [7]:
final_df.createOrReplaceTempView("ViewProduct")

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 9, Finished, Available, Finished)

###### **<mark>Inserting data from View to gold_product using merge query</mark>**

In [8]:
%%sql
MERGE INTO Ecommerce_project.gold_Product as gp 
USING ViewProduct as vp
on gp.Product = vp.Product and gp.Product_Category=vp.Product_Category
WHEN MATCHED THEN

UPDATE SET
gp.Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
INSERT
(
gp.Product_ID,
gp.Product,
gp.Product_Category,
gp.Created_TS,
gp.Modified_TS

)
VALUES
(
vp.Product_ID,
vp.Product,
vp.Product_Category,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP()
)

StatementMeta(, 7c6a270f-fb21-4d4f-963f-48c7ccfae4ac, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>